**Resumo: Extração de *Features* de dados de áudio da ESC50**

---

Nesse documento, se encontram as *features* extraídas de um conhecido conjunto de áudio chamado ESC50. O dataset pode ser encontrado no [github karolpiczak](https://github.com/karolpiczak/ESC-50)

A maioria das *features* selecionadas é proveniente do spectrograma de cada
exemplo do conjunto de dados utilizado.

Devido à natureza dos dados, acredito que seja possível extrair *features*
relacionadas com séries temporais. Inicialmente, utilizarei os métodos usados neste [repositório.](https://github.com/karolpiczak/ESC-50)

**Início da construção do DataFrame**
* Esquema desejado: `{filename, fold, target, category, esc10, src_file, take}`

esse markdown vai para a funnção que cria o esquema

# Importando bibliotecas

In [78]:
from datetime import datetime
import librosa
import librosa.display
import multiprocessing as mp
import numpy as np
import os
import pandas as pd
import re
from scipy.signal import butter, filtfilt
import warnings

import sys
sys.path.append('./python-som/')

#Fazer um markdown só para o padrão
#Criação de um padrão através de ReGex
#ER: [TUDO][GDIGITO]_[G6DIGITOS]-[GCARACTER]-[G2DIGITOS][TUDO][END OF STRING]
#groups()[0]:[GDIGITO]    --> FOLD
#groups()[1]:[G6DIGITOS]  --> CLID_ID
#groups()[2]:[GCARACTER]  --> TAKE
#groups()[3]:[G2DIGITOS]  --> TARGET
PATTERN: str = (r"^.*(\d)-(\d{1,6})-([A-Z])-(\d{1,2}).*$")

**Padrão elaborando usando Expressão Regular (ER):**
* Expressão Regular: `r"^.*(\d)-(\d{1,6})-([A-Z])-(\d{1,2})\.wav$"`
  * Exemplo de string que dá match com a ER: `"5-263902-A-36"`
* Grupos da expressão regular:
  * `groups()[0]`: grupo de um digito, representando `{FOLD}`
  * `groups()[1]`: grupo de seis digito, representando `{CLID_ID}`
  * `groups()[2]`: grupo de uma palavra, representando `{TAKE}`
  * `groups()[3]`: grupo de dois digito, representando `{TARGET}`
* Para entender melhor, acessar [regexr](https://regexr.com/) ou [regex101](https://regex101.com/)

# Definição de funções

## ```extract_data_from_filename (fpath: str) -> list```


- `extract_data_from_filename`: dado o nome bruto do arquivo de áudio, separa
as informações em datatime (com data e horário), nome bruto do arquivo, ala e
grupo
- Parâmetros:
  * `@fpath`: nome bruto do arquivo
- Saida:
  * `@data`: uma lista contendo o datatime, nome bruto do arquivo, ala e grupo 
referente ao áudio

In [134]:
def extract_data_from_filename(fpath: str) -> list:
    #Matches: lista de objetos matches que representam nomes de arquivos que respeitam o padrão
    #Objeto match: a string mas dividida segundo algum padrão
    #re.fullmatch(): retorna um objeto match se toda a string respeitar o padrão. Se não, retorna None
      #Parâmetro pattern: ER definindo o padrão
      #Parâmetro fpath: string a ser testada    
    match: re.Match = re.fullmatch(PATTERN, fpath)

    #Resgata o nome do arquivo
    #matches[i].groups()[j]: acessa o grupo j do objeto matches[i] : match
    #matches.groups()[j]: acessa o grupo j do objeto match : match
    filename : str = "-".join( [i for i in match.groups()] )

    #Constrói as primeiras quatro colunas (metadados) do áudio
    data : list =  [  fpath,             #CAMINHO DO ARQUIVO
                      filename,          #FILENAME
                      match.groups()[0], #FOLD
                      match.groups()[3], #TARGET
                      match.groups()[1], #SRC_FILE OU CLIP_ID
                      match.groups()[2]  #TAKE
                    ]

    return data

## `extract_feature_means(audio_file_path: str, verbose: bool = True) -> pd.DataFrame`

- `extract_feature_means`: carrega um áudio e extrai as features típicas de
processamento de sinal deste áudio (spectral centroid, spectral rollof, etc.)
- Parâmetros:
  * `@audio_file_path`: caminho do arquivo de áudio que se deseja extrair as features
  * `@verbose = True`: indica se haverá print ou não
- Saida:
  * `@df`: pandas.DataFrame contendo todas as features extraídas do áudio

In [135]:
def extract_feature_means(audio_file_path: str, verbose: bool = True) -> pd.DataFrame:
  #Define alguns valores importantes para extração
  if verbose:
      print("File:", audio_file_path)
  #DÚVIDA: QUEM E PORQUE FOR DETERMINADO O 20, 2048 E 512
  #DÚVIDA: ESCREVER AQUI O QUE É CADA UM
  number_of_mfcc = 20 #pensar em diminuir ou aumentar (testar)
  n_fft = 2048  # FFT window size
  hop_length = 512  # number audio of frames between STFT columns

  #Extrai os metadados deste áudio com a função criada anteriormente
  if verbose:
      print("0.Extracting info from filepath...")
  filepath, filename, fold, target, src_file, take = extract_data_from_filename(audio_file_path)

  #Carrega o áudio com librosa
  if verbose:
      print("1.Importing file with librosa...")
  try:
      y, sr = librosa.load(audio_file_path)
  except Exception as e:
      print(e)
      return None

  #librosa.effects.trim(): Corta o silêncio inicial e final de um sinal de áudio
    #Retorna signal: série temporal do sinal cortado
    #Retorna o intervalo não silencioso do sinal
  signal, _ = librosa.effects.trim(y)

  #Armazena em d_audio a magnitude da frequencia do sinal após aplicar o STFT
  #lr.stft(x): Short-time Fourier transform (STFT) que representa o sinal no domínico do tempo e frequencia
    #DÚVIDA: PORQUE O ABS RETORNA A MAGNITUDE E O ANGLE A FASE? NÃO ENTENDI
    #np.abs(D[..., f, t]): função magnitude da frequencia *******
    #np.angle(D[..., f, t]): função fase da frequencia
  if verbose:
      print("2.Fourier transform...")
  #Short-time Fourier transform (STFT)
  d_audio = np.abs(librosa.stft(signal, n_fft = n_fft, hop_length = hop_length))

  #Armazena em db_audio, após transformar a magnitude da frequencia, a unidade comparativa dB do sinal
  #lr.amplitude_to_db(): converte a amplitude do espectrogram para dB
    #np.abs(lr.stft(x)): amplitude
    #ref = np.max: DÚVIDA NÃO ENTENDI ISSO AQUIIIIIII
      #OBS:np.max: pega o maior valor de um eixo dado
  if verbose:
      print("3.Spectrogram...")
  db_audio = librosa.amplitude_to_db(d_audio, ref=np.max)

  #Armazena em s_db_audio, após extrair o espectograma em escala mel, a unidade comparativa dB do sinal
  #librosa.feature.melspectrogram(): computa o espectograma em escala mel do sinal
    #Parâmetro signal: serie temporal do audio
    #Parâmtro sr: taxa de amostragem
  if verbose:
      print("4.Mel spectograms...")
  s_audio = librosa.feature.melspectrogram(signal, sr=sr)
  s_db_audio = librosa.amplitude_to_db(s_audio, ref=np.max)

  #Captura o elemento harmômico e percusivo do sinal
  #librosa.effects.hpss(): decompõem uma serie temporal de audio componentes harmônicos e percursivos
    #Retorno y_harm: serie temporal do elemento harmonico
    #Retorno y_perc: serie temporal do elemento percursivo
  if verbose:
      print("6.Harmonics and perceptrual...")
  y_harm, y_perc = librosa.effects.hpss(signal)

  #Captura as features relacionadas a spectral centroid
  #librosa.feature.spectral_centroid(): computa o spectral centroid
    #Parâmetro signal: serie temporal do audio
    #Parâmtro sr: taxa de amostragem
    #Retorna: centroidnp.ndarray [shape=(…, 1, t)]
  #ibrosa.feature.delta(): computa delta features (estimativa local de derivadas dos dados de entrada)
    #Parâmetro order: a ordem da derivada (primeira, segunda, etc.)
  #DÚVIDA: POR QUE AS DERIVADAS SÃO IMPORTANTES MESMO?
  if verbose:
      print("7.Spectral centroid...")
  # Calculate the Spectral Centroids
  spectral_centroids = librosa.feature.spectral_centroid(signal, sr=sr)[0]
  spectral_centroids_delta = librosa.feature.delta(spectral_centroids)
  spectral_centroids_accelerate = librosa.feature.delta(spectral_centroids, order=2)

  #Captura o cromagrama
  #librosa.feature.chroma_stft(): computa o cromagrama da onda
    #Parâmetro signal: audio time series
    #Parâmetro sr: sample rate
    #Parâmetro hop_length: FFT window size
    #Retorna: um np.ndarray() com a "energia normalizada" para cada croma em cada frama
    #DÚVIDA: não entendi muito bem o que seria energia normalizada
  if verbose:
      print("8.Chroma features...")
  chromagram = librosa.feature.chroma_stft(signal,
                                            sr=sr,
                                            hop_length=hop_length)
  #DÚVIDA: QUE FEATURE É ESSA?
  #librosa.beat.beat_track(): detecta o BPM (MAS O QUE É ISSO?)
    #Retorna: float relativo ao tempo global estimado (batidas por minuto)
    #Retorna: np.ndarr0ay relativo a posição estimada dos eventos de batida (em geral, unidade dado em frames)
  if verbose:
      print("9.Tempo BPM...")
  tempo_y, _ = librosa.beat.beat_track(signal, sr=sr)

  #Captura Spectral Rollof, Spectral Flux e Spectral Bandwidth
  if verbose:
      print("10.Spectral rolloff...")
  #-->Spectral RollOff Vector
  #librosa.feature.spectral_rolloff():
    #Retorna: np.ndarray relativo a frequencia roll-off para cada frame
  #???? ler
  #DÚVIDA: POR QUE PEGAMOS APENAS O FRAME 0?
  spectral_rolloff = librosa.feature.spectral_rolloff(signal, sr=sr)[0]
  #-->Spectral Flux
  #DÚVIDA: O QUE É
  """
  relaciondo ao inicio de um som
  """
  #librosa.onset.onset_strength(): compute o spectral flux "onset strength envelope"
    #Retorna: vetor contendo onset stregth "envelopado"
  onset_env = librosa.onset.onset_strength(y=signal, sr=sr)
  #-->Spectral Bandwidth
  #librosa.feature.spectral_bandwidth(): computa a p-ésima ordem do spectal bandwith
    #Returno: np.ndarray relativo a frequencia bandwidth para cada frame
  spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(signal, sr=sr)[0]
  spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(signal,
                                                            sr=sr,
                                                            p=3)[0]
  spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(signal,
                                                            sr=sr,
                                                            p=4)[0]

#columns = ['filepath', 'fold', 'target', 'src_file', 'take']
  #Formatação das colunas e linhas do DataFrame
  audio_features = {
      "filename":
      filename,
      "filepath":
      filepath,
      "fold":
      fold,
      "target":
      target,
      "src_file":
      src_file,
      "take":
      take,
      #librosa.feature.zero_crossing_rate(): computa a taxa de "zero-crossing" de uma serie temporal de audio
        #Returna: np.ndarray relativo a fração de "zero-crossing" em cada frame
      "zero_crossing_rate":
      np.mean(librosa.feature.zero_crossing_rate(signal)[0]),
      "zero_crossings":
      #librosa.zero_crossings(): encontra "zero-crossing" do sinal 
        #Parâmetro pad = False: não considera signal[0] como sendo válido
        #Retorna: np.ndarray indicando "zero-crossing" do sinal ao longo do eixo
      np.sum(librosa.zero_crossings(signal, pad=False)),
      "spectrogram":
      np.mean(db_audio[0]),
      "mel_spectrogram":
      np.mean(s_db_audio[0]),
      "harmonics":
      np.mean(y_harm),
      "perceptual_shock_wave":
      np.mean(y_perc),
      "spectral_centroids":
      np.mean(spectral_centroids),
      "spectral_centroids_delta":
      np.mean(spectral_centroids_delta),
      "spectral_centroids_accelerate":
      np.mean(spectral_centroids_accelerate),
      "chroma1":
      np.mean(chromagram[0]),
      "chroma2":
      np.mean(chromagram[1]),
      "chroma3":
      np.mean(chromagram[2]),
      "chroma4":
      np.mean(chromagram[3]),
      "chroma5":
      np.mean(chromagram[4]),
      "chroma6":
      np.mean(chromagram[5]),
      "chroma7":
      np.mean(chromagram[6]),
      "chroma8":
      np.mean(chromagram[7]),
      "chroma9":
      np.mean(chromagram[8]),
      "chroma10":
      np.mean(chromagram[9]),
      "chroma11":
      np.mean(chromagram[10]),
      "chroma12":
      np.mean(chromagram[11]),
      "tempo_bpm":
      tempo_y,
      "spectral_rolloff":
      np.mean(spectral_rolloff),
      "spectral_flux":
      np.mean(onset_env),
      "spectral_bandwidth_2":
      np.mean(spectral_bandwidth_2),
      "spectral_bandwidth_3":
      np.mean(spectral_bandwidth_3),
      "spectral_bandwidth_4":
      np.mean(spectral_bandwidth_4),
  }

  #Extraí o DataFrame contendo o MFCC do áudio e suas derivadas primeira e segunda
  mfcc_df = extract_mfcc_feature_means(filename,
                                        signal,
                                        sample_rate=sr,
                                        number_of_mfcc=number_of_mfcc)

  #Converte o dicionário em DataFrame
  df = pd.DataFrame.from_records(data=[audio_features])

  #pd.merge(): fundi objetos DataFrames/Series com memso estilo de database
    #Parâmetro df e mfcc_df: os dois DataFrames que queremos juntas
    #Parâmetro on: a coluna que será usada como orientação para juntar
  df = pd.merge(df, mfcc_df, on = 'filename')

  if verbose:
      print("DONE:", audio_file_path)

  return df

  # librosa.feature.mfcc(signal)[0, 0]

## `extract_mfcc_feature_means(audio_file_name: str, signal: np.ndarray, sample_rate: int, number_of_mfcc: int) -> pd.DataFrame`

- `extract_mfcc_feature_means`: extraí a feature Mel-Frequency Cepstrum Coefficients (MFCC) de um arquivo de áudio, bem como sua derivado primeira e segunda. Por fim, adiciona esses valores em um DataFrame
- Parâmetros:
  * `@audio_file_name`: nome do arquivo de áudio
  * `@signal`: serie temporal representando o áudio
  * `@sample_rate`: taxa de amostragem o áudio
  * `@number_of_mfcc`: NÃO SEI O QUE É ISSO
- Saida:
  * `@df`: pandas.DataFrame contendo as MFCC features do áudio

In [136]:
def extract_mfcc_feature_means(audio_file_name: str, 
                               signal: np.ndarray,
                               sample_rate: int,
                               number_of_mfcc: int) -> pd.DataFrame:
    
    #Computa MFCC de uma serie temporal de áudio, bem como a feature delta de ordem 1 e 2 relacionado a ela
    #librosa.feature.mfcc(): computa MFCC de uma série de áudio
      #Retorna: np.ndarray relativo à sequencia MFCC
      #DÚVIDA: O QUE CADA ELEMENTO DA SEQUÊNCIA QUER DIZER?
    mfcc_alt = librosa.feature.mfcc(y=signal,
                                    sr=sample_rate,
                                    n_mfcc=number_of_mfcc)
    delta = librosa.feature.delta(mfcc_alt)
    accelerate = librosa.feature.delta(mfcc_alt, order=2)

    #Cria o dicionário inicial para formação do DataFrame
    mfcc_features = {
        "filename": audio_file_name,
    }

    #DÚVIDA: DA ONDE SAIU A IDEIA DE CONSIDERAR A DERIVADA PRIMERIA E SEGUNDA NA ANÁLISE
    #Adiciona cada esquema e elemento do MFCC e seus deltas ao dicionário mfcc_features
    for i in range(0, number_of_mfcc):
        # dict.update({'key3': 'geeks'})

        #Adiciona mfcc[i] coefficient ao dicionário
        #string.join(): função embutida nas variáveis tipo string que 
          #junta elementos de uma lista separando-os pela string
        key_name = "".join(['mfcc', str(i)])
        mfcc_value = np.mean(mfcc_alt[i])
        #dictionary.update({key_name: value}): adiciona mais um par ao dicionário
        mfcc_features.update({key_name: mfcc_value})

        #Adiciona mfcc[i] delta coefficient ao dicionário
        key_name = "".join(['mfcc_delta_', str(i)])
        mfcc_value = np.mean(delta[i])
        mfcc_features.update({key_name: mfcc_value})
        
        #Adiciona mfcc[i] acclerate coefficient ao dicionário
        key_name = "".join(['mfcc_accelerate_', str(i)])
        mfcc_value = np.mean(accelerate[i])
        mfcc_features.update({key_name: mfcc_value})

    #pandas.DataFrame.from_records(data): converte estrutura ou registro ndarray em DataFrame
    df = pd.DataFrame.from_records(data=[mfcc_features])

    return df

# Extraindo áudio bruto dos arquivos `.wav` usando librosa

**Acessando caminho dentro do drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

**Definindo início e extensão do arquivo**

In [162]:
EXTENSION: str = '.wav'
COMMON_PATH: str = '/home/tarcidio/analysis-ESC50/'

NUM_ITERATION = '4'

DATA_NAME = 'audios/audio' + NUM_ITERATION
#DATA_NAME = 'audios'
DATA_DIR: str = os.path.join(COMMON_PATH, DATA_NAME)

CSV_DIR = os.path.join(COMMON_PATH, 'csv_feature')
CSV_NAME = 'feature' + NUM_ITERATION + '.csv'


**Varrendo todos os arquivos de um diretório. Para cada arquivo `.wav` encontrado, anexar na lista `fpaths`**
  * Biblioteca `os`: provê funções para mexer no sistema operacional  
  * `os.path.join()`: junta um ou mais componentes de caminho usando `\`
    * Parâmetro `root`: caminho até o arquivo
    * Parâmetro `file`: arquivo em si (podendo ter mais um pouco de caminho)
  * `os.walk()`: gera os nomes dos arquivos dado uma árvore de diretório
    * Parâmetro `DATA_DIR`: diretório raiz
    * Retorna:
      * `dirpath`: caminho do diretório raiz da iteração
      * `dirnames`: caminho dos subdiretórios do diretório raiz da iteração
      * `filenames`: nome do arquivo do diretório
  * `.endswith()`: retorna True se a string terminar com o que é especificado no parâmetro da função

In [163]:
fpaths: list = sorted([
    os.path.join(root, file) 
      for root, _, files in os.walk(DATA_DIR)
        for file in files if file.endswith(EXTENSION)
])

**Sobre o dataset a convenção de nome**

*  2000 registros de audios em formato WAV (5 segundos, 44.1 kHx, mono)
>* Convention: `{FOLD}-{CLIP_ID}-{TAKE}-{TARGET}.wav`
>  * `{FOLD}` - index of the cross-validation fold,
>  * `{CLIP_ID}` - ID of the original Freesound clip,
>  * `{TAKE}` - letter disambiguating between different fragments from the same Freesound clip,
>  * `{TARGET}` - class in numeric format [0, 49].

**Verificando caminhos que respeitam o padrão:**
* Objetos `match`: string, mas dividida segundo algum padrão
* `re.fullmatch()`: retorna um objeto `match` se toda a string respeitar o padrão. Se não, retorna None
  * Parâmetro `pattern`: padrão definido por ER
  * Parâmetro `fpath`: string a ser testada
* `matches[i].groups()[j]`: acessa o grupo `j` de um objeto `matches[i]`

In [164]:
#Matches: lista de objetos matches que representam nomes de arquivos que respeitam o padrão
matches: list = [re.fullmatch(PATTERN, fpath) for fpath in fpaths]

**Confirmando se todos os arquivos respeitam o padrão:**
* Declaração `"raise"`: gera um erro e para o fluxo do programa
  * `raise ValueError()`: tipo de exceção que é gerada quando uma operação ou função recebe um argumento com valor inapropriada

In [165]:
if len(matches) != len(fpaths):
    raise ValueError("check fpath patterns")

In [13]:
#Na função set_index, verify_integrity = false indica que não haverá checagem de index duplicados
#Seta 'filename' como sendo o index
#OBS: por que não usou verify_integrity = False no 'audio_kdd.py'?

#NÃO VAMO SETAR INDEX COM FILENAME AINDA PORQUE USAMOS A COLUNA COMO ENTRADA DE OUTRA FUNÇÃO
#audios.set_index(['filename'], drop = True, inplace = True, verify_integrity=False)

**Preparação para extração das features usando multiprocessamento**

In [166]:
print('Processing', len(fpaths), 'audios...')

#Vetor cujos elementos serão, cada um, a linha do DataFrame final contendo os metadados e as features
result = []

#Variável que determina em quantos processos serão feitas as extrações
#DÚVIDA: POR QUE FOI ESCOLHIDO A QUANTIDADE 32?
n_processes = 4

Processing 18 audios...


**Extração dos features**
* Declaração `with`: usada em manipulações de exceções para manter
o código mais legível. Também simplifica o gerenciamento de recursos
de streams. Ex: abertura de arquivos

1) Sem `with`
```python
  file = open('file_path', 'w')
  try:
      file.write('hello world')
  finally:
      file.close()
```
2) Com `with`
```python
  with open('file_path', 'w') as file:
      file.write('hello world !')
```
  * **NOTE**: com `with`, o próprio Python libera os recursos no caso de erro]
  * **MAIS**: https://www.geeksforgeeks.org/with-statement-in-python/
* `warnings.catch_warnings()`: suprime erros. Em geral, é desejados quando já se
sabe que uma função obsoleta está sendo usada e não quer ver os avisos que elas 
irão gerar. Use esta função após o `with` e com `warnings.simplefilter("ignore")`.

**Multiprocessamento**
* Biblioteca `multiprocessing`: pacote que suporte geração de processos, oferecendo concorrência entre subprocessos. 
* Objeto `Pool`: objeto base usado pela biblitoeca para paralelizar execução de 
uma função em vários valores de entrada, distribuindo os dados de entrada entre
os processos. A classe `Pool`, em suma, representa um ramo de processos ativos
* `mp.Pool(n_processes)`: cria `n_processes` objetos `pool` que vão rodar em paralelo
* `pool.map(function, iterable, chunksize)`: executa a função `function` em cada
um dos elementos da lista `iterable`. `Chunksize` determina quantos desses elementos serão distribuidos para cada um dos processsos.
* `pool.terminate()`: para o processo ativo imediatamente 
* `pool.close()`: espera pelo fim do processo ativo
* `pool.join()`: impede que mais tarefas sejam subtidas ao pool. Quando todas forem concluídas, encerasse o `mp.Pool`
* MAIS: https://docs.python.org/3/library/multiprocessing.html

In [167]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    with mp.Pool(processes=n_processes) as pool:
        """result recebe, de vários processamentos, elementos que contem os metadados
e as features de cada arquivo de áudio. A variável result será usada como
argumento para concatenar todas as informações e gerar o DataFrame"""
        result = pool.map(extract_feature_means, #Função a ser aplicada a cada nome de arquivo
                          iterable=fpaths,     #Lista com o nome dos arquivos
                          chunksize=len(fpaths) // n_processes
                                                 #Distribuição para cada processo
                          ) 
        pool.close()
        pool.join()

File:File:File:File:    /home/tarcidio/analysis-ESC50/audios/audio4/1-155858-A-25.wav/home/tarcidio/analysis-ESC50/audios/audio4/2-135649-C-45.wav/home/tarcidio/analysis-ESC50/audios/audio4/5-156999-B-19.wav/home/tarcidio/analysis-ESC50/audios/audio4/3-170383-A-38.wav


0.Extracting info from filepath...0.Extracting info from filepath...0.Extracting info from filepath...



1.Importing file with librosa...1.Importing file with librosa...1.Importing file with librosa...

0.Extracting info from filepath...
1.Importing file with librosa...

2.Fourier transform...
3.Spectrogram...
4.Mel spectograms...
6.Harmonics and perceptrual...
2.Fourier transform...
3.Spectrogram...
4.Mel spectograms...
2.Fourier transform...
3.Spectrogram...
4.Mel spectograms...
7.Spectral centroid...
6.Harmonics and perceptrual...
8.Chroma features...
9.Tempo BPM...
6.Harmonics and perceptrual...
10.Spectral rolloff...
2.Fourier transform...
3.Spectrogram...
4.Mel spectograms...
DONE: /home/tarcidio/analysis-ESC50/a

**Concatena os dados extraídos**
* `enumerate()`:  adiciona um contador para um iteravel e retorna na forma de objeto enumerador

Exemplo:

```python
l1 = ["eat", "sleep", "repeat"]

for ele in enumerate(l1):
    print (ele)
```

Output:

```python
(0, 'eat')
(1, 'sleep')
(2, 'repeat')
```

Exemplo:

```python
for count, ele in enumerate(l1):
  print(count)
  print(ele)
```

Output:

```python  
0
eat
1
sleep
2
repeat
```



In [168]:
print("Done processing audios. Concatenating and writing to output file...")
for idx, i in enumerate(result):
    if i is None:
        del result[idx]
#pandas.concat(): concatena objetos pandas
audios_means : pd.DataFrame = pd.concat(result)
output_path : str = os.path.join(CSV_DIR, CSV_NAME)
audios_means.to_csv(output_path, index = False)
print("End")

Done processing audios. Concatenating and writing to output file...
End
